# TensorFlow Recommenders: Quickstart

In this tutorial, we build a simple matrix factorization model using the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) with TFRS. We can use this model to recommend movies for a given user.

# Import TensorFlow Recommender System (TFRS)

In [4]:
!pip install -q sagemaker==2.9.2
!pip install -q sagemaker-experiments==0.1.24
!pip install -q tensorflow==2.3.0
!pip install -q tensorflow-recommenders==0.2.0
!pip install -q tensorflow-datasets==4.0.0

In [5]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# Load Ratings

In [6]:
ratings = tfds.load('movielens/100k-ratings',                     
                    download=False,
                    data_dir='./tensorflow_datasets/',
                    split='train')
print(ratings)

AssertionError: Dataset movielens: could not find data in ./tensorflow_datasets/. Please make sure to call dataset_builder.download_and_prepare(), or pass download=True to tfds.load() before trying to access the tf.data.Dataset object.

In [ ]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
print(ratings)

# Load Movies

In [ ]:
movies = tfds.load('movielens/100k-movies', 
                   download=False,                   
                   data_dir='./tensorflow_datasets/',
                   split='train')
print('Movies BEFORE', movies)

In [ ]:
movies = movies.map(lambda x: x["movie_title"])
print('Movies AFTER', movies)

# Create Vocabularies
Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:

In [ ]:
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

In [ ]:
print(user_ids_vocabulary.get_vocabulary())

# Create the Model

We can define a TFRS model by inheriting from `tfrs.Model` and implementing the `compute_loss` method:

In [ ]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_embedding: tf.keras.Model,
      movie_embeddings: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_embeddings = user_embeddings
    self.movie_embeddings = movie_embeddings

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed using the retrieval task
    user_embeddings = self.user_embeddings(features['user_id'])
    movie_embeddings = self.movie_embeddings(features['movie_title'])

    return self.task(user_embeddings, movie_embeddings)

# Define User and Movie Models
Define the two models.

In [ ]:
user_embeddings = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 128)
])

movie_embeddings = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 128)
])

# Define the Retrieval Task

In [ ]:
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_embeddings)
  )
)

# Train the Retrieval Model

In [ ]:
model = MovieLensModel(user_embeddings, movie_embeddings, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

In [ ]:
# Train.
model.fit(ratings.batch(4096), epochs=1)

# Make Predictions
Use brute-force search to set up retrieval using the trained representations.

In [ ]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_embeddings)
index.index(movies.batch(100).map(model.movie_embeddings), movies)

In [ ]:
import os

local_model_dir_bruteforce_model = './exported_models/bruteforce_model/'

tensorflow_saved_model_path_bruteforce_model = os.path.join(local_model_dir_bruteforce_model, 'tensorflow/saved_model/0')

os.makedirs(tensorflow_saved_model_path_bruteforce_model, exist_ok=True)

In [ ]:
k = 5
user_id = "42"

_, titles = index(np.array([user_id]))

print(f"Top {k} recommendations for user {user_id}: {titles[0, :k]}")

In [ ]:
print('Compiled model {}'.format(index))          
print(index.summary())

In [ ]:
index.save(tensorflow_saved_model_path_bruteforce_model, save_format='tf')

In [ ]:
!saved_model_cli show --all --dir $tensorflow_saved_model_path_bruteforce_model

The following works!


```
!saved_model_cli run --input_exprs 'input_1=[str(42)]' --tag_set serve --signature_def serving_default --dir $tensorflow_saved_model_path_bruteforce_model
```

```
Result for output key output_1:
[[3.9453535 2.9104383 2.7285933 2.6687438 2.5107827 2.4038887 2.3974428
  2.386409  2.321732  2.3211298]]
Result for output key output_2:
[[b'Rent-a-Kid (1995)' b'Last Dance (1996)'
  b'Adventures of Pinocchio, The (1996)'
  b'Winnie the Pooh and the Blustery Day (1968)'
  b'Aristocats, The (1970)' b'Celtic Pride (1996)'
  b'Conan the Barbarian (1981)' b'House Arrest (1996)'
  b'Just Cause (1995)' b'Johnny 100 Pesos (1993)']]
```

In [ ]:
!saved_model_cli run --input_exprs 'input_1=["$user_id"]' --tag_set serve --signature_def serving_default --dir $tensorflow_saved_model_path_bruteforce_model

# All in one cell

In [ ]:
import time
import random
import pandas as pd
from glob import glob
import pprint
import argparse
import json
import subprocess
import sys
import os

# Set the directory you want to start from
#for dirName, subdirList, fileList in os.walk('/etc/ssl/certs/'):
#    print('Found directory: %s' % dirName)
#    for fname in fileList:
#        print('\t%s' % fname)        
        
from typing import Dict, Text
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import numpy as np


class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_embedding: tf.keras.Model,
      movie_embeddings: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_embeddings = user_embeddings
    self.movie_embeddings = movie_embeddings

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed using the retrieval task
    user_embeddings = self.user_embeddings(features['user_id'])
    movie_embeddings = self.movie_embeddings(features['movie_title'])

    return self.task(user_embeddings, movie_embeddings)


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
     
    args, _ = parser.parse_known_args()
    print("Args:") 
    print(args)
    
    env_var = os.environ 
    print("Environment Variables:") 
    pprint.pprint(dict(env_var), width = 1) 

    local_model_dir = './model' # os.environ['SM_MODEL_DIR']
    output_dir = './output' # args.output_dir
    print('output_dir {}'.format(output_dir))    

    train_data = './tensorflow_datasets/' # args.train_data
    print('train_data {}'.format(train_data))
    epochs = 1 # args.epochs
    print('epochs {}'.format(epochs))    
    learning_rate = 0.5 # args.learning_rate
    print('learning_rate {}'.format(learning_rate))    
    enable_tensorboard = False # args.enable_tensorboard
    print('enable_tensorboard {}'.format(enable_tensorboard))       
    dataset_variant = '100k' # args.dataset_variant
    print('dataset_variant {}'.format(dataset_variant))
    embedding_dimension = int(256) # int(args.embedding_dimension)
    print('embedding_dimension {}'.format(embedding_dimension))       

    # Load the ratings data to use for training
    ratings = tfds.load('movielens/{}-ratings'.format(dataset_variant), 
                        download=False,
                        data_dir=train_data,
                        split='train')
    print('Ratings raw', ratings)

    # Transform the ratings data specific to our training task
    ratings = ratings.map(lambda x: {
        'movie_title': x['movie_title'],
        'user_id': x['user_id']
    })
    print('Ratings transformed', ratings)    

    # Load the movies data to use for training
    movies = tfds.load('movielens/{}-movies'.format(dataset_variant),
                       download=False,
                       data_dir=train_data,
                       split='train')
    print('Movies raw', movies)
    
    # Transform the movies data specific to our training task
    movies = movies.map(lambda x: x['movie_title'])
    print('Movies transformed', movies)

    # Create the user vocabulary and user embeddings
    user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
    user_ids_vocabulary.adapt(ratings.map(lambda x: x['user_id']))

    user_embeddings = tf.keras.Sequential([
        user_ids_vocabulary,
        tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(),
                                  embedding_dimension)
    ])

    # Create the movie vocabulary and movie embeddings
    movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
    movie_titles_vocabulary.adapt(movies)

    movie_embeddings = tf.keras.Sequential([
        movie_titles_vocabulary,
        tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(),
                                  embedding_dimension)
    ])

    # Specify the task and the top-k metric to optimize during model training
    task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
        movies.batch(128).map(movie_embeddings)
    ))

    # Define the optimizer and hyper-parameters
    optimizer = tf.keras.optimizers.Adagrad(learning_rate)
    print('Optimizer:  {}'.format(optimizer))

    # Setup the callbacks to use during training
    callbacks = []

    # Setup the Tensorboard callback if Tensorboard is enabled
    if enable_tensorboard: 
        # Tensorboard Logs 
        tensorboard_logs_path = os.path.join(local_model_dir, 'tensorboard/')
        os.makedirs(tensorboard_logs_path, exist_ok=True)

        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tensorboard_logs_path)
        print('Adding Tensorboard callback {}'.format(tensorboard_callback))
        callbacks.append(tensorboard_callback)
    print('Callbacks: {}'.format(callbacks))

    # Create a custom Keras model with the user embeddings, movie embeddings, and optimization task
    model = MovieLensModel(user_embeddings, movie_embeddings, task)
    
    # Compile the model and prepare for training
    model.compile(optimizer=optimizer)

    # Train the model
    model.fit(ratings.batch(4096), epochs=epochs)

    # Make some sample predictions to test our model
    # Note:  This is required to save and server our model with TensorFlow Serving
    #        See https://github.com/tensorflow/tensorflow/issues/31057 for more  details.
    index = tfrs.layers.factorized_top_k.BruteForce(query_model=model.user_embeddings)
    index.index(movies.batch(100).map(model.movie_embeddings), movies)

    user_id = '42'
    _, titles = index(np.array([user_id]))

    k = 10
    print(f'Top {k} recommendations for user {user_id}: {titles[0, :k]}')

    # Print a summary of our recommender model
    print('Trained index {}'.format(index))
    print(index.summary())

    # Save the TensorFlow SavedModel for Serving Predictions
    # SavedModel Output
    tensorflow_saved_model_path = os.path.join(local_model_dir,
                                               'tensorflow/saved_model/0')
    os.makedirs(tensorflow_saved_model_path, exist_ok=True)
    
    print('tensorflow_saved_model_path {}'.format(tensorflow_saved_model_path))
    index.save(tensorflow_saved_model_path, save_format='tf')

In [ ]:
user_id = "42"

!saved_model_cli run --input_exprs 'input_1=np.array(["$user_id"])' --tag_set serve --signature_def serving_default --dir ./model/tensorflow/saved_model/0